# Academy of Py

Three observable trends from this dataset:

    1.Score results are negatively correlated with school size.
    2.Spending levels are also negatively correlated with scores.
    3.Charter schools seem to be better even when the size effect is factored in.  The largest Charter school (2200 students) had much better scores than the smallest District school (2900). Unfortunately the data has only large District and small Charter school size so the two factors can't be differentiated.

### District Summary

In [26]:
import pandas as pd

students = "raw_data/students_complete.csv"
schools = "raw_data/schools_complete.csv"

pd.options.display.float_format = '{:,.2f}'.format


# District Numbers

# Read-in students and schools csv:
students_df = pd.read_csv(students)
schools_df = pd.read_csv(schools)

# Find Totals for district:
school_list = students_df['school'].unique()
total_schools = len(school_list)
total_students = students_df['name'].count()
total_budget = schools_df['budget'].sum()

# Find averages for scores:
ave_math_score=students_df['math_score'].mean()
ave_reading_score=students_df['reading_score'].mean()

# Locate and count passing (i.e. better than 69) scores
math_pass_df = students_df.loc[students_df["math_score"] >= 70, :]
math_pass_count = math_pass_df['math_score'].count()
reading_pass_df = students_df.loc[students_df["reading_score"] >= 70, :]
reading_pass_count = reading_pass_df['reading_score'].count()

# Calculate percentages:
percent_pass_math = math_pass_count/total_students*100
percent_pass_reading = reading_pass_count/total_students*100
percent_pass_overall = (math_pass_count + reading_pass_count)/total_students*50

# Build district summary dataframe:
district_breakdown = pd.DataFrame({"Total Schools": [total_schools],
                                   "Total Students": [total_students],
                                   "Total Budget": [total_budget],
                                   "Average Math Score": [ave_math_score],
                                   "Average Reading Score": [ave_reading_score],
                                   "% Passing Math":[percent_pass_math],
                                   "% Passing Reading":[percent_pass_reading],
                                   "% Overall Passing Rate": [percent_pass_overall]})
district_breakdown['Total Budget'] = district_breakdown['Total Budget'].map('${:,.2f}'.format)
district_breakdown=district_breakdown[['Total Schools','Total Students','Total Budget','Average Math Score', 'Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]

district_breakdown



,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.99,81.88,74.98,85.81,80.39


### School Summary

In [15]:
# Prepare schools df for merge with average students scores:
schools_df = schools_df.rename(index=str, columns={"name":"school"})
school_groups_df =students_df.groupby(["school"])
school_score_groups_df = school_groups_df[['school','math_score','reading_score']].mean().reset_index()

# Merge schools data with average scores data:
aves_merge_df = pd.merge(school_score_groups_df, schools_df, on="school")

# Find pass counts by school and merge with school summary df:
school_math_pass_group  = math_pass_df.groupby(["school"])
school_math_pass_df = school_math_pass_group[['math_score']].count().reset_index()
school_reading_pass_group  = reading_pass_df.groupby(["school"])
school_reading_pass_df = school_reading_pass_group[['reading_score']].count().reset_index()
pass_count_merge_df = pd.merge(aves_merge_df, school_math_pass_df, on="school")
pass_count_merge_df = pd.merge(pass_count_merge_df, school_reading_pass_df, on="school")

# Complete per school summary with percentages columns and per student budgets:
pass_count_merge_df['% Passing Math'] = pass_count_merge_df['math_score_y']/pass_count_merge_df['size']*100
pass_count_merge_df['% Passing Reading'] = pass_count_merge_df['reading_score_y']/pass_count_merge_df['size']*100
pass_count_merge_df['% Overall Passing'] = (pass_count_merge_df['% Passing Math'] + pass_count_merge_df['% Passing Reading'])/2
pass_count_merge_df['Per Student Budget'] = pass_count_merge_df['budget']/pass_count_merge_df['size']

school_summary = pass_count_merge_df.rename(index=str, columns={"school":"School",'math_score_x':'Average Math Score',
                                                                'reading_score_x':'Average Reading Score','type':'School Type',
                                                                'size':'Total Students','budget':'Total Budget'})

school_summary = school_summary.drop(['math_score_y','reading_score_y','School ID'], axis = 1)
school_summary = school_summary[["School",'School Type','Total Students','Total Budget','Per Student Budget',
                                 'Average Math Score', 'Average Reading Score','% Passing Math',
                                 '% Passing Reading','% Overall Passing']]
school_summary = school_summary.set_index('School')
school_summary['Total Budget'] = school_summary['Total Budget'].map('${:,.2f}'.format)
school_summary['Per Student Budget'] = school_summary['Per Student Budget'].map('${:,.2f}'.format)
school_summary

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,74.31
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.59
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.36
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,73.80
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.27
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,73.81
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,94.38
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,73.64


### Top Performing Schools by Passing Rate

In [16]:
ranked_schools = school_summary.sort_values(by=['% Overall Passing'], ascending=False)
ranked_schools.head()


,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.59
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,95.29
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,95.27
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.27
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,95.20


### Low Performing Schools by Passing Rate

In [17]:
ranked_schools.tail()


,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School,,,,,,,,,
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,73.80
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,73.64
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.50
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.36
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,73.29


### Math Scores by Grade

In [21]:
# Complete scores by grade tables:
students_9th_df=students_df.loc[students_df['grade'] == '9th',:]
students_10th_df=students_df.loc[students_df['grade'] == '10th',:]
students_11th_df=students_df.loc[students_df['grade'] == '11th',:]
students_12th_df=students_df.loc[students_df['grade'] == '12th',:]

students_9th_df=students_9th_df.rename(index=str, columns={"school":"School"})
students_10th_df=students_10th_df.rename(index=str, columns={"school":"School"})
students_11th_df=students_11th_df.rename(index=str, columns={"school":"School"})
students_12th_df=students_12th_df.rename(index=str, columns={"school":"School"})



# Groupbys for each grade

students_9th_df = students_9th_df.set_index('School')
students_10th_df = students_10th_df.set_index('School')
students_11th_df = students_11th_df.set_index('School')
students_12th_df = students_12th_df.set_index('School')

grade_group_9th_df = students_9th_df.groupby(['School'])
math_scores_9th_df = grade_group_9th_df['math_score'].mean()

grade_group_10th_df = students_10th_df.groupby(['School'])
math_scores_10th_df = grade_group_10th_df['math_score'].mean()
grade_group_11th_df = students_11th_df.groupby(['School'])
math_scores_11th_df = grade_group_11th_df['math_score'].mean()
grade_group_12th_df = students_12th_df.groupby(['School'])
math_scores_12th_df = grade_group_12th_df['math_score'].mean()

reading_scores_9th_df = grade_group_9th_df['reading_score'].mean()
reading_scores_10th_df = grade_group_10th_df['reading_score'].mean()
reading_scores_11th_df = grade_group_11th_df['reading_score'].mean()
reading_scores_12th_df = grade_group_12th_df['reading_score'].mean()


math_scores_by_grade = pd.DataFrame({'9th':math_scores_9th_df,"12th":math_scores_12th_df,'11th':math_scores_11th_df,
                                     '10th':math_scores_10th_df, })
math_scores_by_grade = math_scores_by_grade[['9th','10th','11th','12th']]
math_scores_by_grade



,9th,10th,11th,12th
School,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


### Reading Scores by Grade

In [22]:
reading_scores_by_grade = pd.DataFrame({"12th":reading_scores_12th_df,'11th':reading_scores_11th_df,
                                     '10th':reading_scores_10th_df, '9th':reading_scores_9th_df})
reading_scores_by_grade = reading_scores_by_grade[['9th','10th','11th','12th']]
reading_scores_by_grade



,9th,10th,11th,12th
School,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


### Scores by School Spending

In [23]:
# Compare results based on per student spending:
spend_bins = [550,590,630,670]
spend_labels = ["Low ($550-590)","Medium ($591-630)","High  >  $630"]
spend_categories = pd.cut(pass_count_merge_df["Per Student Budget"], spend_bins, labels=spend_labels)
breakdown_by_spending_df = pd.DataFrame({'Spend Level':spend_categories,
                                        'Average Math Score':aves_merge_df['math_score'],                                       
                                        'Average Reading Score':aves_merge_df['reading_score'],
                                        '% Passing Math':pass_count_merge_df['% Passing Math'], 
                                        '% Passing Reading':pass_count_merge_df['% Passing Reading'],
                                        'Overall Passing Rate':pass_count_merge_df['% Overall Passing']})
breakdown_by_spending_df.sort_values('Spend Level')
grouped_breakdown_by_spending_df = breakdown_by_spending_df.groupby(['Spend Level'])
spending_per_student = grouped_breakdown_by_spending_df.mean()
spending_per_student = spending_per_student[['Average Math Score', 'Average Reading Score', '% Passing Math',
                                             '% Passing Reading', 'Overall Passing Rate']]
spending_per_student


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spend Level,,,,,
Low ($550-590),83.46,83.93,93.46,96.61,95.04
Medium ($591-630),81.90,83.16,87.13,92.72,89.93
High > $630,77.87,81.37,70.35,83.00,76.67


### Scores by School Size

In [24]:
# Compare results based on school size:
school_size_bins =[400,1900,3500,5000]
school_size_labels = ['Small (400-1900)', 'Medium (1901-3500)', 'Large >3500']
school_size_categories = pd.cut(pass_count_merge_df["size"], school_size_bins, labels=school_size_labels)
breakdown_by_size_df = pd.DataFrame({'School Size':school_size_categories,
                                        'Average Math Score':aves_merge_df['math_score'],                                       
                                        'Average Reading Score':aves_merge_df['reading_score'],
                                        '% Passing Math':pass_count_merge_df['% Passing Math'], 
                                        '% Passing Reading':pass_count_merge_df['% Passing Reading'],
                                        'Overall Passing Rate':pass_count_merge_df['% Overall Passing']})
grouped_breakdown_by_size_df = breakdown_by_size_df.groupby(['School Size'])
size_comparison = grouped_breakdown_by_size_df.mean()
size_comparison = size_comparison[['Average Math Score', 'Average Reading Score', '% Passing Math',
                                   '% Passing Reading', 'Overall Passing Rate']]
size_comparison


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (400-1900),83.50,83.88,93.59,96.59,95.09
Medium (1901-3500),78.43,81.77,73.46,84.47,78.97
Large >3500,77.06,80.92,66.46,81.06,73.76


### Scores by School Type

In [25]:
# Compare results based on school type:
school_types = schools_df[['type','school']]
calcs_by_type_df = pd.DataFrame({'school':aves_merge_df['school'],
                                        'Average Math Score':aves_merge_df['math_score'],                                       
                                        'Average Reading Score':aves_merge_df['reading_score'],
                                        '% Passing Math':pass_count_merge_df['% Passing Math'], 
                                        '% Passing Reading':pass_count_merge_df['% Passing Reading'],
                                        'Overall Passing Rate':pass_count_merge_df['% Overall Passing']})
breakdown_by_type_df = pd.merge(calcs_by_type_df, school_types,on='school')
breakdown_by_type_df = breakdown_by_type_df.rename(index=str, columns={"type":"Type"})
grouped_breakdown_by_type_df = breakdown_by_type_df.groupby(['Type'])
chart_vs_dist_df = grouped_breakdown_by_type_df.mean()
chart_vs_dist_df = chart_vs_dist_df[['Average Math Score', 'Average Reading Score','% Passing Math',
                                     '% Passing Reading','Overall Passing Rate']]
chart_vs_dist_df



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type,,,,,
Charter,83.47,83.90,93.62,96.59,95.10
District,76.96,80.97,66.55,80.80,73.67
